In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage
from mesostat.stat.anova import as_pandas_lst, anova_homebrew

from lib.sych.data_fc_db_raw import DataFCDatabase
from lib.sych.plot_helper import imshow_dataset_by_mouse, imshow_dataset_by_session
from lib.sych.metric_helper import metric_by_session

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
ds = DataStorage('sych_result_individual_region.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
print(dataDB.mice)
print(dataDB.dataTypes)
print(dataDB.trialTypeNames)

# 1. Noise
## 1.1 SNR via AutoCorrelation

**Plan:**
* AutoCorr[1] is a number that is close to 1 if data is changing slowly, and close to zero if data changes very fast.

**FIXME**:
* Remove Performance above 1
* If performance becomes important, discuss with Y whether to use original or pooled performance, as they differ significantly

In [ ]:
metricName = 'autocorr_d1'
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    for datatype in dataDB.get_data_types():
        dataName = metricName + '_' + datatype
        print(dataName)
        metric_by_session(dataDB, mc, ds, mousename, metricName, 'p', datatype=datatype, dataName=dataName)

In [ ]:
metricName = 'autocorr_d1'
for datatype in dataDB.get_data_types():
    dataName = metricName + '_' + datatype
    imshow_dataset_by_mouse(dataDB, ds, dataName, limits=[0,1])

# 2. Variance

* Across sessions
    - Explained by performance
* Across channels, trials, timesteps
    - Explained by trial type

In [ ]:
mousename = 'mvg_4'
trialTypeNames = dataDB.get_trial_type_names()
sessions = dataDB.get_sessions(mousename)

rezLst = []
for session in sessions:
    print(session)
    dataLst = [
        dataDB.get_neuro_data({'session' : session}, datatype='bn_session', trialType=trialType)[0]
        for trialType in trialTypeNames
    ]

    colNames = ('trials', 'timesteps', 'channels')
    dfThis = as_pandas_lst(dataLst, colNames, trialTypeNames, 'trialType')
    dfRez = anova_homebrew(dfThis, 'rez')
    display(dfRez)
    rezLst += [list(dfRez['F'])[1:-1]]

In [ ]:
sessions = dataDB.get_sessions(mousename)

names = list(dfRez['axis'])[1:-1]
rezArr = np.array(rezLst).T

fig, ax = plt.subplots()
ax.set_ylabel('mean_ssq')
for name, x in zip(names, rezArr):
    plt.semilogy(x, label=name)
    
ax.set_xticks(np.arange(len(sessions)))
ax.set_xticklabels(sessions, rotation=90)
ax.legend()
plt.show()


# 2. Activity

## 2.1 Variance-based measures

In [ ]:
for metricName in ['mean', 'std', 'avg_entropy']:
    for iMouse, mousename in enumerate(sorted(dataDB.mice)):
        for datatype in dataDB.get_data_types():
            dataName = metricName + '_' + datatype
            print(dataName)
            metric_by_session(dataDB, mc, ds, mousename, metricName, 'p', datatype=datatype, dataName=dataName)

In [ ]:
for metricName in ['mean', 'std', 'avg_entropy']:
    for datatype in dataDB.get_data_types():
        dataName = metricName + '_' + datatype
        imshow_dataset_by_mouse(dataDB, ds, dataName)

# 3. Temporal Structure
## 3.1 Mean vs Time

In [ ]:
metricName = 'mean'
for iMouse, mousename in enumerate(sorted(dataDB.mice)):
    for datatype in dataDB.get_data_types():
        dataName = metricName + '_time_' + datatype
        if datatype == 'raw':
            zscoreDim='rs'
        else:
            zscoreDim=None
        
        metric_by_session(dataDB, mc, ds, mousename, metricName, 'ps',
                          datatype=datatype,
                          dataName=dataName,
                          zscoreDim=zscoreDim)

In [ ]:
metricName = 'mean'
for datatype in dataDB.get_data_types(mousename):
    dataName = metricName + '_time_' + datatype
    print(dataName)
    
    imshow_dataset_by_session(dataDB, ds, dataName, limits=None,
                              fig1size=(12, 2), colBased=False, aspect='auto')